# Генеративные сети

Рекуррентные нейронные сети (RNN) и их более сложные разновидности, такие как сети долговременной кратковременной памяти (LSTM) и GRU, дают нам механизм моделирования языка, то есть они могут выучивать порядок слов и предоставлять прогнозы для следующего слова в последовательности. Это позволяет нам использовать RNN для **генеративных задач**, таких как генерация обычного текста, машинный перевод и даже субтитры к изображениям.

В архитектуре RNN, которую мы обсуждали в предыдущем разделе, каждый блок RNN производил следующее скрытое состояние в качестве выхода. Тем не менее, мы также можем добавить еще один выход к каждой рекуррентной единице, что позволит нам породить **последовательность** (которая равна по длине исходной последовательности). Более того, мы можем использовать такие блоки RNN, которые не принимают вход на каждом шаге, а просто берут некоторый вектор начального состояния, а затем производят последовательность выходов.

В этом примере мы рассмотрим простую генеративную модель. Для простоты используем посимвольную токенизацию и будем генерировать текст буква за буквой. Во время обучения нам нужно взять некоторый текстовый корпус и разбить его на последовательности букв. 

In [12]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Создание словаря и токенизация

Чтобы построить генеративную сеть на уровне символов, нам нужно разделить текст на отдельные символы, а не на слова. Слой `TextVectorization`, который мы использовали раньше, не может этого сделать, поэтому у нас есть варианты:

* Вручную загрузить текст и выполнить токенизацию, как в [этом официальном примере Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Использовать класс `Tokenizer` для токенизации на уровне символов.

Мы пойдем по второму пути. `Tokenizer` также может использоваться для токенизации по словам, поэтому у нас будет возможность довольно легко переключаться с посимвольного уровня токенизации на уровень слов.

Чтобы выполнить токенизацию на уровне символов, нам нужно передать параметр `char_level=True`:

In [ ]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Мы также хотим использовать один специальный токен для обозначения **конца последовательности**, который мы будем называть `<eos>`. Добавим его вручную в словарь:

In [ ]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Теперь, чтобы закодировать текст в последовательность чисел, мы можем использовать такой код:

In [ ]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Обучение генеративной RNN для генерации заголовков

Способ, которым мы будем обучать RNN генерировать заголовки новостей, заключается в следующем. На каждом шаге мы возьмем один заголовок, который будет подан на вход RNN, и для каждого входного символа мы попросим сеть сгенерировать следующий за ним символ:

![Image showing an example RNN generation of the word 'HELLO'.](./images/rnn-generate.png)

Для последнего символа нашей последовательности мы попросим сеть сгенерировать `<eos>`.

Важное отличие генеративных RNN заключается в том, что мы получаем выходные данные на каждом шаге RNN, а не только от конечной ячейки. Этого можно достичь, указав параметр `return_sequences` ячейке RNN.

Таким образом, во время обучения вход сети представляет собой последовательность закодированных символов некоторой длины, а выход - последовательность той же длины, но сдвинутую на один элемент и заканчивающуюся '<eos>'. Минибатч будет состоять из нескольких таких последовательностей, и нам нужно будет использовать **padding** для выравнивания всех последовательностей.

Давайте создадим функцию, подготавливающую такой минибатч. Мы будем сначала разбивать датасет на минибатчи с помощью `.batch()`, а затем применять `map` для преобразования. Функция ниже будет сразу преобразовывать целый минибатч:

In [ ]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Вот что мы делаем в этой функции:

* Сначала извлекаем текст из строкового тензора. Поскольку строка представлена в байтовом представлении, преобразуем её в unicode с помощью `decode`
* `text_to_sequences` преобразует список строк в список целочисленных тензоров
* `pad_sequences` дополняет эти тензоры до их максимальной длины
* Наконец, мы применяем к символам one-hot encoding - это первый из возвращаемых результатов, входные последовательности
* Для получения выходных последовательностей мы убираем первый символ, добавляем в конец токен `<eos>`

Тонкость состоит в том, что эта функция является **питонической** (*Pythonic*), т.е. она не может быть автоматически переведена в вычислительный граф Tensorflow. Мы получим ошибки, если попытаемся использовать эту функцию непосредственно в функции `Dataset.map`. Нам нужно обернуть этот вызов с помощью оболочки `py_function`: 

In [ ]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Примечание**: Различие между преобразованиями данных на стороны Python или Tensorflow может показаться слишком сложным, и вы можете задаться вопросом, почему мы не преобразуем набор данных с помощью стандартных функций Python, прежде чем передать его в `fit`. Хотя это определенно можно сделать, использование `Dataset.map` имеет огромное преимущество, потому что конвейер преобразования данных выполняется с использованием вычислительного графа Tensorflow, который использует преимущества вычислений GPU и сводит к минимуму необходимость передачи данных между CPU / GPU.

Теперь мы можем построить нашу генераторную сеть и начать обучение. Она может быть основана на любой из рассмотренных нами RNN-архитектур (простая, LSTM или GRU), но лучше всё-таки использовать LSTM.

Поскольку сеть принимает символы в качестве входных данных, а размер словарного запаса довольно мал, нам не нужен слой эмбеддингов, будем подавать one-hot-encoded вход напрямую на вход ячейке LSTM. Выходной слой будет представлять собой классификатор `Dense`, который преобразует выходные данные LSTM в номера токенов в словаре, а точнее в распределение вероятностей появления таких токенов.

Кроме того, поскольку мы имеем дело с последовательностями переменной длины, мы можем использовать слой `Masking` для указания той части последовательности, которую нужно игнорировать. Это не очень критично, потому что нас не очень интересует все, что выходит за рамки токена `<eos>`, но мы будем использовать его ради получения некоторого опыта работы с этим слоем. `input_shape` будет `(None, vocab_size)`, где `None` указывает на последовательность переменной длины, а выход будет иметь размер `(None,vocab_size)`,  как вы можете видеть из `summary`:

In [ ]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, None, 84)          0         
                                                                 
 lstm (LSTM)                 (None, None, 128)         109056    
                                                                 
 dense (Dense)               (None, None, 84)          10836     
                                                                 
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________


2022-12-04 01:51:51.761621: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-04 01:51:52.502525: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-04 01:51:56.147378: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


15000/15000 [==============================] - 1655s 110ms/step - loss: 1.5458


## Генерация выходных данных

Теперь, когда мы обучили модель, мы можем использовать её для генерации текста. Прежде всего, нам нужен способ декодирования текста, представленного последовательностью номеров токенов. Для этого мы могли бы использовать функцию `tokenizer.sequences_to_texts`; однако она плохо работает с токенизацией на уровне символов. Поэтому мы возьмем словарь токенов из токенизатора (называемый `word_index`), построим обратную карту и напишем собственную функцию декодирования:

In [ ]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Теперь давайте займемся генерацией. Мы начнем с некоторой строки `start`, закодируем ее в последовательность `inp`, а затем на каждом шаге будем вызывать нашу сеть, чтобы вывести следующий символ. 

Выход сети `out` представляет собой вектор элементов размером `vocab_size`, представляющих собой вероятности для каждого токена, и мы можем найти наиболее вероятный номер токена, используя `argmax`. Затем мы добавляем этот символ в сгенерированный список токенов и переходим к следующему шагу генерации. Этот процесс генерации одного символа повторяется `size` раз, чтобы сгенерировать необходимое количество символов, и мы заканчиваем досрочно, когда встречаем `eos`.

In [ ]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s to buy to buy to start of the streated in Iraq'

## Выборка выходных данных во время обучения 

Поскольку в случае с генерацией у нас нет никаких простых метрик, таких как *точность*, мы можем наблюдать за тем, как наша модель становится лучше, делая **выборку**, т.е. генерируя некоторую строку во время обучения. Для этого мы будем использовать такую возможность Keras как **обратные вызовы** (*callbacks*), т.е. функции, которые мы передаем функции `fit`, и которые будут периодически вызываться во время обучения. 

In [ ]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
  299/15000 [..............................] - ETA: 27:58 - loss: 1.3183

KeyboardInterrupt: 

Этот пример уже генерирует довольно хороший текст, но его можно дополнительно улучшить несколькими способами:

* **Больше данных**. Мы использовали только заголовки новостей для нашей задачи, но вы можете поэкспериментировать с полным текстом новостей. Помните, что RNN не слишком хороши в обработке длинных последовательностей, поэтому имеет смысл либо разбить их на короткие предложения, либо всегда тренироваться на фиксированной длине последовательности некоторого предопределенного значения `num_chars` (скажем, 256). Вы можете попытаться изменить приведенный выше пример на такую архитектуру, используя [официальный учебник Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) в качестве вдохновения.

* **Многослойный LSTM**. Имеет смысл попробовать 2 или 3 слоя ячеек LSTM. Как мы уже упоминали в предыдущем разделе, каждый слой LSTM извлекает определенные шаблоны из текста, и в случае генератора, работающего на уровне символов, мы можем ожидать, что более низкий уровень LSTM будет отвечать за извлечение слогов, а более высокие уровни - за слова и словосочетания. Это может быть просто реализовано путем передачи параметра `number-of-layer` в конструктор LSTM.

* Вы также можете поэкспериментировать с **блоками GRU** и посмотреть, какие из них работают лучше, и с **различными размерами скрытых слоев**. Слишком большой скрытый слой может привести к переобучению (например, сеть будет генерировать на выходе точный текст из входного датасета), а меньший размер может не дать хорошего результата.

## Температура генерации

В предыдущем определении функции `generate` мы всегда брали символ с наибольшей вероятностью в качестве следующего символа в сгенерированном тексте. Это приводило к тому, что текст часто зацикливался, т.е. циклически перемещался между одними и теми же последовательностями символов снова и снова, как в этом примере:

```
today of the second the company and a second the company ...
```

Однако, если мы посмотрим на распределение вероятностей для следующего символа, то может оказаться, что разница между несколькими топовыми вероятностями не так велика, например, один символ может иметь вероятность 0.2, другой - 0.19 и т.д. Например, при поиске следующего символа в последовательности **play**, следующим символом одинаково хорошо может быть либо пробел, либо **e** (как в слове **player**).

Это приводит нас к выводу, что не всегда «справедливо» выбирать символ с максимальной вероятностью, потому что выбор второго по величине может в равной степени привести нас к содержательному тексту. Разумнее делать **выборку** символов из словаря в соответствии с распределением вероятностей, полученным на выходе сети.

Эта выборка может быть выполнена с помощью функции `np.multinomial`, которая реализует так называемое **мультиномиальное распределение**. Функция, которая реализует такую **мягкую** генерацию текста, определена ниже:

In [ ]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Мы ввели еще один параметр под названием **температура**, который используется, чтобы указать, насколько сильно мы должны придерживаться наибольшей вероятности. Если температура равна 1.0, мы делаем справедливую мультиномиальную выборку, а когда температура уходит в бесконечность - все вероятности становятся равными, и мы случайным образом выбираем следующий символ. В приведенном ниже примере мы можем наблюдать, что текст становится бессмысленным, когда мы слишком сильно повышаем температуру, и он напоминает «циклический» жестко сгенерированный текст, когда температура становится ближе к 0. 